## INTRODUCTION

Nous sommes analystes marketing chez Books Online, une importante librairie en ligne spécialisée dans les livres
d'occasion. Dans le cadre denvos fonctions,nvous essaonsz de suivre manuellement les prix des livres d'occasion sur
les sites web de vos concurrents, mais cela représente trop de travail en vous n'arronsez pas à y faire face : il y a
trop de livres et trop de librairies en ligneN! Vous nt votre équipeonsvez décidé d'automatiser cette tâche laborie se
via un programme (un scraper) développé en Python, capable d'extraire les informations tarifaires d'au res
librairies en liNous sommess êtes schargé de développer une version bêta de ce système pour suivre les prix des livres chez Books to
Scrape, un revendeur de livres encessaires

## 1. LIBRAIRIES

Mais tout d'abord, nous allons importer les libraries qui nous seront nécessaires

In [1]:
#Insertion de toutes les librairies que nous utiliserons
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

## 2. PARTIE SCRAPING

Nous allons maintenant récupérer la liste de tous les livres qui sont dans le site.
Vu aue tout le travail a été regroupé dans une seule cellule nous allons d'abord expliquer en détail le contenu du code. 

#### 2.1. Création de fonction pour extraire les données d'une page

Une fonction get_page_data() permettra de recueillir les données d'une page du site concurrent. Elle prend en argument l'URL de la fenêtre en question et récupére l'ensembles des classes concernant les livres.

Par la suite, elle crée une base vide "livres" qui gardera les informations recueillies. Les informations seront le titre, le prix (ainsi que la devise), la note (l'évaluation) et la disponibilté du livre, tout cela en receuillant l'ensemble des classes correspondant à chaque information. 

#### 2.2. Création de la fonction principale pour le scraping

Avant cela, nous avons gardé dans une variable l'url de la page d'acceuil. Nous avons également crée la base vide qui contiendra la totalité des livres du site.

Ensuite nous recupérons les informations de la page d'acceuil afin d'y plonger et récupérer la liste de l'ensemble des catégories existantes dans ce site. 

Après avoir obtenu la liste nous commençons par éliminer la catégorie Books qui n'est pas une catégorie en soit, puis nous nous plaçcons au niveau de la page de la 1ère catégorie. Le lien de cette catégorie a été formée en étudiant la configuration des liens pour chaque catégorie qui suit une certaine logique. Par la suite nous avons utilisé la fonction permettant de récupérer les données d'une page puis nous avons ajouté ces informations dans la base "tous_les_livres". Nous formons ensuite le lien de la page suivante vu que la configuration de celle ci suit également une certaine logique, puis nous utilisons ce lien pour refaire le même processus que la page précédente en donnant le nouveau lien obtenu. 

Certains contrôles ont été fait afin de de permettre des changements de pages fluides et des liens corrects.

Après quoi, nous passons à la catégorie suivante et nous reprenons le même processus jusqu'à parcourir toute la liste des catégories.

Le code se présente comme suit:


In [2]:
i=2
def get_page_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        liste_livres = soup.find_all('article', class_='product_pod')

        # Liste pour stocker les informations de chaque livre dans cette page
        livres = []

        # Parcourir tous les livres de cette page et récupérer les informations souhaitées
        for livre in liste_livres:
            titre = livre.h3.a['title']
            prix = livre.find('p', class_='price_color').text
            note = livre.find('p', class_='star-rating')['class'][1]
            disponibilite = livre.find('p', class_='instock availability').text.strip()

            # Ajouter les informations du livre à la liste 'livres'
            livres.append({
                'Catégorie': current_category,
                'Titre': titre,
                'Prix': prix,
                'Note': note,
                'Disponibilité': disponibilite
            })

        return livres
    else:
        print("La requête a échoué avec le statut :", response.status_code)
        return []

# URL de la page de la page d'accueil
url_page = "http://books.toscrape.com/catalogue/page-1.html"

# Liste pour stocker les informations de tous les livres de toutes les catégories
tous_les_livres = []

# Récupérer le contenu de la page d'acceuil
response = requests.get(url_page)

# Vérifier si la requête a réussi (statut 200)
if response.status_code == 200:
    # Analyser le contenu HTML de la page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupérer la liste des catégories de la page d'accueil
    liste_categories = soup.find('ul', class_='nav-list').find_all('a')
    liste_categories.pop(0)
    # Parcourir toutes les catégories et récupérer les informations pour chaque catégorie
    for categorie in liste_categories:
        j=1
        #récupérer le nom de catégorie concernée
        current_category = categorie.text.strip()
        #mettre le nom en miniscule vu que les liens du sites utilisent ces noms en miniscule
        current_category = current_category.lower()
        #remplacer les espaces par des "-" vu que les liens fonctionnent ainsi également
        current_category=current_category.replace(' ','-')
        i1=str(i)
        print(current_category)
        #former l'url de la catégorie
        url_categorie = "http://books.toscrape.com/catalogue/category/books/"+current_category+"_"+i1+"/index.html"
        i=i+1
        # Récupérer les informations pour la catégorie actuelle et toutes ses pages
        while True:
            
            # Récupérer les informations pour la page actuelle
            livres_pages = get_page_data(url_categorie)
            tous_les_livres.extend(livres_pages)
        
            # Trouver le lien de la page suivante
            response = requests.get(url_categorie)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                next_page = soup.find('li', class_='next')
                if next_page is None:
                    # S'il n'y a pas de lien vers la page suivante, sortir de la boucle
                    break
                else:
                    # Construire l'URL de la page suivante
                    j=j+1
                    j1=str(j)
                    url_categorie = "http://books.toscrape.com/catalogue/category/books/"+current_category+"_"+i1+"/page-"+j1+".html"
                    
            else:
                print(f"La requête pour la catégorie '{current_category}' a échoué avec le statut :", response.status_code)
                break

travel
mystery
historical-fiction
sequential-art
classics
philosophy
romance
womens-fiction
fiction
childrens
religion
nonfiction
music
default
science-fiction
sports-and-games
add-a-comment
fantasy
new-adult
young-adult
science
poetry
paranormal
art
psychology
autobiography
parenting
adult-fiction
humor
horror
history
food-and-drink
christian-fiction
business
biography
thriller
contemporary
spirituality
academic
self-help
historical
christian
suspense
short-stories
novels
health
politics
cultural
erotica
crime


------------------------------
------------------------------
------------------------------

On peut vérifier le travail en vérifiant la taille de la base. Nous avons exactement le même nombre de livres que celui présent sur le site.

In [3]:
len(tous_les_livres)

1000

## 3. AFFICHAGE ET ARRANGEMENT DE LA BASE

In [4]:
base_finale=pd.DataFrame(tous_les_livres)

In [5]:
base_finale.head()

,Catégorie,Titre,Prix,Note,Disponibilité
0,travel,It's Only the Himalayas,£45.17,Two,In stock
1,travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,£49.43,Four,In stock
2,travel,See America: A Celebration of Our National Par...,£48.87,Three,In stock
3,travel,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,Two,In stock
4,travel,Under the Tuscan Sun,£37.33,Three,In stock


### Changer les valeurs de la colonne "note" par leur équivalent en chiffre

Nous allons transformer les valeurs de la colonne note pour les avoir en chiffres afin de faciliter les interprétations futures.

In [6]:
chiffre={ "One":1, "Two":2, "Three":3, "Four":4, "Five":5}

base_finale["Note"]=[chiffre[val] for val in base_finale["Note"]]

In [7]:
base_finale.head()

,Catégorie,Titre,Prix,Note,Disponibilité
0,travel,It's Only the Himalayas,£45.17,2,In stock
1,travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,£49.43,4,In stock
2,travel,See America: A Celebration of Our National Par...,£48.87,3,In stock
3,travel,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,2,In stock
4,travel,Under the Tuscan Sun,£37.33,3,In stock


### Convertir les prix pour qu'ils soient tous de la même unité

Certains prix sont en euros et d'autres en pounds. Nous devons donc les convertir pour qu'ils aient tous la même unité pour pouvoir faire la comparaison

In [8]:
#créer une fonction qui garde les nombres
def prix_val (prix):
    return re.sub(r'[^\d.]','',prix)
#Une autre qui garde la devise
def prix_devise(prix):
    return re.sub(r'[0-9.]','',prix)

#Créer les variables séparées de la valeur et de la devise   
for i in range (0, len(base_finale)):
    base_finale.loc[i,"Prix_euro"]=prix_val(base_finale.loc[i,"Prix"])
    base_finale.loc[i,"devises"]=prix_devise(base_finale.loc[i,"Prix"])

#Convertir le type de la colonne des valeurs
base_finale["Prix_euro"]=base_finale["Prix_euro"].astype(float)

#Calculer l'équivalent en euro
for i in range (0, len(base_finale)):
    if (base_finale.loc[i,"devises"]=="£"):
        base_finale.loc[i,"Prix_euro"]=round(int(base_finale.loc[i,"Prix_euro"])/0.86,2)

base_finale=base_finale.drop(columns=["devises"])

In [9]:
base_finale.head()

,Catégorie,Titre,Prix,Note,Disponibilité,Prix_euro
0,travel,It's Only the Himalayas,£45.17,2,In stock,52.33
1,travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,£49.43,4,In stock,56.98
2,travel,See America: A Celebration of Our National Par...,£48.87,3,In stock,55.81
3,travel,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,2,In stock,41.86
4,travel,Under the Tuscan Sun,£37.33,3,In stock,43.02


In [11]:
base_finale.to_csv("base_finale.csv")